In [17]:
import sys
# import dolfinx
import numpy as np
import matplotlib.pylab as plt
import slepc4py

from scipy.optimize import root
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc

from dolfinx import fem, io, plot, mesh
import ufl

import pyvista
pyvista.start_xvfb()


slepc4py.init(sys.argv)

In [18]:
# Define Geometry
L, B, H = 5., 0.3, 0.6

geometry_mesh = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, H, B])],
    [25, 3, 2],
    cell_type=mesh.CellType.tetrahedron)

In [19]:
# Define vector space from geometry mesh
V = fem.VectorFunctionSpace(geometry_mesh, ("CG", 2))

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [20]:
# Boundary condition
def clamped_boundary(x):
    return np.isclose(x[0], 0)

fdim = geometry_mesh.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(geometry_mesh, fdim, clamped_boundary)

u_D = fem.Function(V)
with u_D.vector.localForm() as loc:
    loc.set(0)

bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets))

In [ ]:
# Define actual problem
E, nu = (2E11), (0.3)
rho = (7850)
mu = E/2./(1+nu)
lambda_ = E*nu/(1+nu)/(1-2*nu)


def epsilon(u):
    return 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)


def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(3) \
           + 2*mu*epsilon(u)

T = fem.Constant(geometry_mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0),
                    PETSc.ScalarType(0)))
k_form = ufl.inner(sigma(u), epsilon(v))*ufl.dx
m_form = rho*ufl.dot(u, v)*ufl.dx
k = fem.form(k_form)
m = fem.form(m_form)


K = fem.petsc.assemble_matrix(k, bcs=[bc2])
K.assemble()

M = fem.petsc.assemble_matrix(m, bcs=[bc2])
M.assemble()

In [ ]:
# Define eigensolver and solve for eigenvalues
no_eigenvalues = 6 

eigensolver = SLEPc.EPS().create(geometry_mesh.comm)
eigensolver.setOperators(K, M)
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GNHEP)
tol = 1e-9
eigensolver.setTolerances(tol=tol)
st = eigensolver.getST()
st.setType(SLEPc.ST.Type.SINVERT)
eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
eigensolver.setTarget(100)
eigensolver.setDimensions(nev=no_eigenvalues)

# Solve the eigensystem
eigensolver.solve()

# Get the number of converged eigenpairs
evs = eigensolver.getConverged()

# Create dummy vectors for the eigenvectors to store the results in
vr, vi = K.createVecs()

In [ ]:
for i in range(evs):
    # e_val = eigensolver.getEigenpair(i, vr, vi)
    # Get the ith eigenvalue and eigenvector (vr and vi are placeholders for
    # the real and complex parts of the eigenvectors) they are then saved in
    # the input variables (vr, vi)
    eigenvalue = eigensolver.getEigenpair(i, vr, vi)
    # e_vec = eigensolver.getEigenvector(i, eh.vector)

    if (~np.isclose(eigenvalue.real, 1.0)):
        # Calculation of eigenfrequency from real part of eigenvalue
        freq_3D = np.sqrt(eigenvalue.real)/2/np.pi

        eigenmode = fem.Function(V)
        eigenmode.vector[:] = vr.getArray()

        """
        # Get coordinates from vector space V
        coord = V.tabulate_dof_coordinates()

        # Get number of degress of freedom and subspaces
        num_sub_spaces = V.num_sub_spaces
        num_dofs_per_component = int(np.size(coord)/ num_sub_spaces)

        # vector = np.zeros((num_sub_spaces, num_dofs_per_component))
        vector = eigenmode.vector.getArray().reshape(num_dofs_per_component, num_sub_spaces)
        
        x = coord[:, 0]
        y = coord[:, 1]
        z = coord[:, 2]
        mode_magnitude = np.sum(np.abs(vector), axis=1)
        """

        # plot_shape_with_resonance(x,y,mode_magnitude, L)
        # Create plotter and pyvista grid
        # p = pyvista.Plotter(off_screen=True)
        p = pyvista.Plotter()
        topology, cell_types, geometry = plot.create_vtk_mesh(V)
        grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)


        # Attach vector values to grid and warp grid by vector
        grid["u"] = eigenmode.x.array.reshape((geometry.shape[0], 3))
        actor_0 = p.add_mesh(grid, style="wireframe", color="k")
        warped = grid.warp_by_vector("u", factor=5)
        actor_1 = p.add_mesh(warped, show_edges=True)
        p.show_axes()
        if not pyvista.OFF_SCREEN:
           p.show()
        else:
            figure_as_array = p.screenshot("deflection.png")

        print(
            "Solid FE: {0:8.5f} [Hz]".format(freq_3D))


Widget(value="<iframe src='http://localhost:46479/index.html?ui=P_0x7f7d8878e920_20&reconnect=auto' style='wid…

Solid FE:  9.83051 [Hz]


Widget(value="<iframe src='http://localhost:46479/index.html?ui=P_0x7f7e1c0ef7c0_21&reconnect=auto' style='wid…

Solid FE: 19.45146 [Hz]


Widget(value="<iframe src='http://localhost:46479/index.html?ui=P_0x7f7da80efcd0_22&reconnect=auto' style='wid…

...

Solid FE: 118.82391 [Hz]


Widget(value="<iframe src='http://localhost:46479/index.html?ui=P_0x7f7d1c026f50_25&reconnect=auto' style='wid…

Solid FE: 165.68699 [Hz]


Widget(value="<iframe src='http://localhost:46479/index.html?ui=P_0x7f7d1c102c50_26&reconnect=auto' style='wid…

Solid FE: 253.16437 [Hz]
